In [54]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd
import skforecast

print(skforecast.__version__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/joaquin/Documents/GitHub/skforecast
0.19.0


In [55]:
import numpy as np
import pandas as pd
import joblib
import nannyml as nml
from sklearn.ensemble import RandomForestRegressor
from skforecast.datasets import fetch_dataset
from skforecast.drift_detection import PopulationDriftDetector

In [67]:
data = joblib.load(r'/home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/tests/tests_population_drift/fixture_data_population_drift.joblib')
data_train = data.copy()
data_new  = data.iloc[len(data) // 2 :].copy()
data_train['weather'] = data_train['weather'].astype('category')
data_new['weather'] = pd.Categorical(data_new['weather'], categories=data_train['weather'].cat.categories)

In [68]:
detector = nml.UnivariateDriftCalculator(
    column_names=data_train.columns.tolist(),
    timestamp_column_name="date_time",
    chunk_period='M',
    categorical_methods=['chi2', 'jensen_shannon'],
    continuous_methods=['kolmogorov_smirnov', 'jensen_shannon'],
)
detector.fit(reference_data=data_train.reset_index())
results_nannyml = detector.calculate(data=data_new.reset_index())
results_nannyml_train = results_nannyml.filter(period='reference').to_df(multilevel=False)
results_nannyml_predict = results_nannyml.filter(period='analysis').to_df(multilevel=False)

In [69]:
detector_sk = PopulationDriftDetector(
        chunk_size='MS',            
        threshold=3,
        threshold_method='std'
    )
detector_sk.fit(data_train)
results_skforecast, _ = detector_sk.predict(data_new)

In [74]:
# Compare statistics learned for training chunks
# ==============================================================================
nannyml_fitted_stats = {'empirical_dist_js_': {},
                        'empirical_dist_ks_': {},
                        'empirical_dist_chi2_': {}
                        }
# Compare the statistic jensen_shannon for training chunks
print("Comparing jensen_shannon statistics for all features")
for feature in data_train.columns:
    print(f'Comparing feature: {feature}')
    skforecast_values = np.array(detector_sk.empirical_dist_js_[feature])
    nanny_ml_values = results_nannyml_train[f'{feature}_jensen_shannon_value'].to_numpy()
    nannyml_fitted_stats['empirical_dist_js_'][feature] = nanny_ml_values
    np.testing.assert_array_almost_equal(skforecast_values, nanny_ml_values, decimal=5)
print("")


# Compare the statistic kolgorov_smirnov for training chunks
print("Comparing kolmogorov_smirnov statistics for numerical features")
for feature in data_train.select_dtypes(include=['number']).columns:
    print(f'Comparing feature: {feature}')
    skforecast_values = np.array(detector_sk.empirical_dist_ks_[feature])
    nanny_ml_values = results_nannyml_train[f'{feature}_kolmogorov_smirnov_value'].to_numpy()
    nannyml_fitted_stats['empirical_dist_ks_'][feature] = nanny_ml_values
    np.testing.assert_array_almost_equal(skforecast_values, nanny_ml_values, decimal=5)
print("")


# Compare the statistic chi2 for training chunks
print("Comparing chi2 statistics for categorical features")
for feature in data_train.select_dtypes(include=['category', 'object']).columns:
    print(f'Comparing feature: {feature}')
    skforecast_values = np.array(detector_sk.empirical_dist_chi2_[feature])
    nanny_ml_values = results_nannyml_train[f'{feature}_chi2_value'].to_numpy()
    nannyml_fitted_stats['empirical_dist_chi2_'][feature] = nanny_ml_values
    np.testing.assert_array_almost_equal(skforecast_values, nanny_ml_values, decimal=5)

from joblib import dump
dump(nannyml_fitted_stats, 'fixture_nannyml_fitted_stats.joblib')

Comparing jensen_shannon statistics for all features
Comparing feature: holiday
Comparing feature: workingday
Comparing feature: weather
Comparing feature: temp
Comparing feature: atemp
Comparing feature: hum
Comparing feature: windspeed
Comparing feature: users
Comparing feature: month
Comparing feature: hour
Comparing feature: weekday

Comparing kolmogorov_smirnov statistics for numerical features
Comparing feature: holiday
Comparing feature: workingday
Comparing feature: temp
Comparing feature: atemp
Comparing feature: hum
Comparing feature: windspeed
Comparing feature: users
Comparing feature: month
Comparing feature: hour
Comparing feature: weekday

Comparing chi2 statistics for categorical features
Comparing feature: weather


['fixture_nannyml_fitted_stats.joblib']

In [75]:
def test_empirical_distributions_match_nannyml():
    """
    Test that the empirical distributions computed by PopulationDriftDetector
    during fit match the ones from NannyML implementation.
    
    This test verifies that the Jensen-Shannon, Kolmogorov-Smirnov, and Chi2
    statistics calculated during the fit phase match the reference values
    computed by NannyML with the same configuration (chunk_size='MS', 
    categorical_methods=['chi2', 'jensen_shannon'], 
    continuous_methods=['kolmogorov_smirnov', 'jensen_shannon']).
    
    The reference values were generated using NannyML 0.13.1 and saved in
    'fixture_nannyml_fitted_stats.joblib'.
    """
    # Fit the detector
    detector_sk = PopulationDriftDetector(
        chunk_size='MS',
        threshold=3,
        threshold_method='std'
    )
    detector_sk.fit(data)
    
    # Load reference statistics from NannyML
    nannyml_fitted_stats = joblib.load('fixture_nannyml_fitted_stats.joblib')
    
    # Compare Jensen-Shannon statistics for all features
    for feature in data.columns:
        skforecast_values = np.array(detector_sk.empirical_dist_js_[feature])
        nannyml_values = nannyml_fitted_stats['empirical_dist_js_'][feature]
        np.testing.assert_array_almost_equal(
            skforecast_values, 
            nannyml_values, 
            decimal=5,
            err_msg=f"Jensen-Shannon statistics mismatch for feature '{feature}'"
        )
    
    # Compare Kolmogorov-Smirnov statistics for numerical features
    for feature in data.select_dtypes(include=['number']).columns:
        skforecast_values = np.array(detector_sk.empirical_dist_ks_[feature])
        nannyml_values = nannyml_fitted_stats['empirical_dist_ks_'][feature]
        np.testing.assert_array_almost_equal(
            skforecast_values, 
            nannyml_values, 
            decimal=5,
            err_msg=f"Kolmogorov-Smirnov statistics mismatch for feature '{feature}'"
        )
    
    # Compare Chi2 statistics for categorical features
    for feature in data.select_dtypes(include=['category', 'object']).columns:
        skforecast_values = np.array(detector_sk.empirical_dist_chi2_[feature])
        nannyml_values = nannyml_fitted_stats['empirical_dist_chi2_'][feature]
        np.testing.assert_array_almost_equal(
            skforecast_values, 
            nannyml_values, 
            decimal=5,
            err_msg=f"Chi2 statistics mismatch for feature '{feature}'"
        )
test_empirical_distributions_match_nannyml()   

In [63]:
from joblib import dump, load
dump(nannyml_fitted_stats, 'fixture_nannyml_fitted_stats.joblib')

['fixture_nannyml_fitted_stats.joblib']

In [60]:
# Compare statistics in prediction chunks
# ==============================================================================
# Compare the statistic jensen_shannon for prediction chunks
print("Comparing jensen_shannon statistics for all features")
for feature in data_new.columns:
    print(f'Comparing feature: {feature}')
    skforecast_values = results_skforecast.query('feature == @feature')['js_statistic'].to_numpy()
    nanny_ml_values = results_nannyml_predict[f'{feature}_jensen_shannon_value'].to_numpy()
    np.testing.assert_array_almost_equal(skforecast_values, nanny_ml_values, decimal=5)
print("")

# Compare the statistic kolgorov_smirnov for prediction chunks
print("Comparing kolmogorov_smirnov statistics for numerical features")
for feature in data_new.select_dtypes(include=['number']).columns:
    print(f'Comparing feature: {feature}')
    skforecast_values = results_skforecast.query('feature == @feature')['ks_statistic'].to_numpy()
    nanny_ml_values = results_nannyml_predict[f'{feature}_kolmogorov_smirnov_value'].to_numpy()
    np.testing.assert_array_almost_equal(skforecast_values, nanny_ml_values, decimal=5)
print("")

# Compare the statistic chi2 for prediction chunks
print("Comparing chi2 statistics for categorical features")
for feature in data_new.select_dtypes(include=['category', 'object']).columns:
    print(f'Comparing feature: {feature}')
    skforecast_values = results_skforecast.query('feature == @feature')['chi2_statistic'].to_numpy()
    nanny_ml_values = results_nannyml_predict[f'{feature}_chi2_value'].to_numpy()
    np.testing.assert_array_almost_equal(skforecast_values, nanny_ml_values, decimal=5)
print("")

Comparing jensen_shannon statistics for all features
Comparing feature: holiday
Comparing feature: workingday
Comparing feature: weather
Comparing feature: temp
Comparing feature: atemp
Comparing feature: hum
Comparing feature: windspeed
Comparing feature: users
Comparing feature: month
Comparing feature: hour
Comparing feature: weekday

Comparing kolmogorov_smirnov statistics for numerical features
Comparing feature: holiday
Comparing feature: workingday
Comparing feature: temp
Comparing feature: atemp
Comparing feature: hum
Comparing feature: windspeed
Comparing feature: users
Comparing feature: month
Comparing feature: hour
Comparing feature: weekday

Comparing chi2 statistics for categorical features
Comparing feature: weather



## Unit Test Created

The comparison code has been converted into a pytest unit test located at:
```
skforecast/drift_detection/tests/tests_population_drift/test_empirical_distributions_match_nannyml.py
```

The test file contains two test functions:

1. **`test_empirical_distributions_match_nannyml()`**: Main test that verifies all empirical distributions (Jensen-Shannon, Kolmogorov-Smirnov, and Chi2) match the NannyML reference values.

2. **`test_empirical_distributions_jensen_shannon_categorical()`**: Focused test specifically for the Jensen-Shannon distance on categorical features to ensure the base=2 parameter is correctly applied.

The fixture file `fixture_nannyml_fitted_stats.joblib` has been saved to the test directory and contains the reference values from NannyML.

Both tests pass successfully ✓